In [24]:
import pandas as pd
import numpy as np

In [25]:
train = pd.read_csv('../data/train_clean.csv',  index_col='Date', parse_dates=['Date'])
spray = pd.read_csv('../data/updated_spray.csv', index_col='Date', parse_dates=['Date'])
weather = pd.read_csv('../data/clean_weather.csv', index_col='Date', parse_dates=['Date'])

In [26]:
train.shape

(9693, 12)

In [27]:
weather.shape

(2944, 20)

In [28]:
spray.head()

,Unnamed: 0,Latitude,Longitude,DateTime
Date,,,,
2011-08-29,0,42.391623,-88.089163,2011-08-29 18:56:58
2011-08-29,1,42.391348,-88.089163,2011-08-29 18:57:08
2011-08-29,2,42.391022,-88.089157,2011-08-29 18:57:18
2011-08-29,3,42.390637,-88.089158,2011-08-29 18:57:28
2011-08-29,4,42.390410,-88.088858,2011-08-29 18:57:38


In [29]:
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [30]:
df = pd.merge(train, weather, on = 'Date')

In [31]:
df.shape

(9693, 50)

In [38]:
df['Day_length'] = (df['Sunset_x'] - df['Sunrise_x'])

In [39]:
df['Day_length'].value_counts()

1466    628
1395    422
1281    404
1512    376
1480    344
1513    330
1453    315
1514    312
1502    292
1501    258
1337    248
1451    212
1219    211
1490    209
1507    188
1177    185
1340    185
1331    185
1297    172
1511    167
1497    165
1494    160
1205    159
1477    157
1202    142
1458    139
1506    137
1318    137
1473    136
1469    131
       ... 
1505    115
1464    112
1275    109
1208    109
1326    106
1399    104
1221     93
1504     92
1199     92
1302     92
1482     90
1329     82
1186     65
1310     62
1188     61
1300     60
1499     59
1495     58
1510     57
1305     54
1323     53
1321     52
1308     48
1350     40
1163     26
1496     25
1449     25
1352     15
1503     13
1462      3
Name: Day_length, Length: 65, dtype: int64

In [32]:
df.to_csv('../data/dataframe.csv')

In [33]:
df.head()

,Unnamed: 0,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,...,Sunrise_y,Sunset_y,CodeSum_y,Depth_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y
Date,,,,,,,,,,,,,,,,,,,,,
2007-05-29,0,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,0,0,BR HZ,0,0.0,29.44,30.09,5.8,16,7.4
2007-05-29,1,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,0,0,BR HZ,0,0.0,29.44,30.09,5.8,16,7.4
2007-05-29,2,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,...,0,0,BR HZ,0,0.0,29.44,30.09,5.8,16,7.4
2007-05-29,3,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,0,0,BR HZ,0,0.0,29.44,30.09,5.8,16,7.4
2007-05-29,4,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,0,0,BR HZ,0,0.0,29.44,30.09,5.8,16,7.4
